<a href="https://colab.research.google.com/github/sh1erkeh/python-homework/blob/main/Copy_of_tasks_RU_gc_rc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача 1:** Изучите как работает базовый reference counting

Создайте строку и исследуйте ее reference count на разных этапах. Верните кортеж: (count_after_creation, count_after_ref, count_after_del)

In [2]:
import sys
import gc
import weakref

In [4]:
def task1():
    my_string = "hello world"
    count_after_creation = sys.getrefcount(my_string)

    another_ref = my_string
    count_after_ref = sys.getrefcount(my_string)

    del another_ref
    count_after_del = sys.getrefcount(my_string)

    return (count_after_creation, count_after_ref, count_after_del)

# Проверка
print("Задача 1:", task1())

Задача 1: (4, 5, 4)


**Задача 2:** Влияние функций на Reference Counting

Создайте список и передайте его в функцию.
Сравните reference count до и после вызова функции.
Верните кортеж: (count_before_call, count_during_call, count_after_call)

In [5]:
def task2():

    def process_list(lst):
        return sys.getrefcount(lst)

    my_list = [1, 2, 3]
    count_before_call = sys.getrefcount(my_list)
    count_during_call = process_list(my_list)
    count_after_call = sys.getrefcount(my_list)

    return (count_before_call, count_during_call, count_after_call)

# Проверка
print("Задача 2:", task2())

Задача 2: (2, 3, 2)


**Задача 3:** Циклические ссылки и Memory Leaks

Создайте два объекта с циклической ссылкой, затем разорвите ее.
Используйте gc для проверки количества собранных объектов.
Верните количество объектов собранных gc после разрыва ссылки.

In [13]:
def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    node1 = Node("a")
    node2 = Node("b")
    node1.ref = node2
    node2.ref = node1

    nodes = [node1, node2]

    node1.ref = None
    node2.ref = None

    del nodes
    del node1
    del node2

    c = gc.collect()

    return c

print("Задача 3:", task3())

Задача 3: 6


**Задача 4:** Сравнение Reference Count для разных типов данных

Сравните reference count для чисел, строк, списков и словарей.
Верните словарь с reference count для каждого типа после создания.

In [17]:
def task4():
    int_obj = 42
    str_obj = "adsgjngra"
    list_obj = [1, 2, 3]
    dict_obj = {'a': 1, 'b': 2}

    r = {
        'int': sys.getrefcount(int_obj),
        'str': sys.getrefcount(str_obj),
        'list': sys.getrefcount(list_obj),
        'dict': sys.getrefcount(dict_obj)
    }
    return r

print("Задача 4:", task4())

Задача 4: {'int': 4294967295, 'str': 3, 'list': 2, 'dict': 2}


**Задача 5:** Weak References

Создайте два объекта со слабой ссылкой (weakref) между ними.
Убедитесь, что объекты могут быть удалены сборщиком мусора.
Верните True если weakref не увеличивает reference count.

In [18]:
def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    data1 = Data(100)
    r_count = sys.getrefcount(data1)

    weak_ref = weakref.ref(data1)
    assert weak_ref() is data1
    return sys.getrefcount(data1) == r_count

print("Задача 5:", task5())

Задача 5: True


**Задача 6:** Мониторинг работы Garbage Collector

Зарегистрируйте callback функцию для отслеживания работы GC.
Верните список событий, которые отследил callback.

In [19]:
def task6():
    events = []

    def gc_callback(phase, info):
        events.append(f"Phase: {phase}, Info: {info}")

    gc.callbacks.append(gc_callback)

    obj = [i for i in range(1000)]
    del obj
    gc.collect()

    gc.callbacks.remove(gc_callback)

    return events

print("Задача 6:", task6())

Задача 6: ["Phase: start, Info: {'generation': 2, 'collected': 0, 'uncollectable': 0}", "Phase: stop, Info: {'generation': 2, 'collected': 43, 'uncollectable': 0}"]


**Задача 7:** Анализ поколений в GC

Создайте объекты и отследите их перемещение между поколениями GC.
Верните кортеж с количеством объектов в каждом поколении до и после создания объектов.

In [21]:
def task7():
    before = gc.get_count()

    objects = []
    for i in range(1000):
        objects.append([j for j in range(100)])

    after = gc.get_count()

    del objects
    gc.collect()

    final = gc.get_count()

    return before, after, final

print("Задача 7:", task7())

Задача 7: ((323, 0, 0), (612, 1, 0), (21, 0, 0))


**Задача 8:** Мониторинг порогов сборки мусора

Изучите как меняются счетчики поколений GC при создании объектов.
Верните словарь с состоянием счетчиков до и после создания объектов.

In [23]:
def task8():
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()

    objects = [[i for i in range(50)] for _ in range(500)]

    after_creation_count = gc.get_count()

    collected = gc.collect(0)
    after_collect_count = gc.get_count()

    del objects
    gc.collect()

    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }

print("Задача 8:", task8())

Задача 8: {'thresholds': (700, 10, 10), 'initial_count': (424, 0, 0), 'after_creation': (210, 1, 0), 'after_collect_gen0': (0, 2, 0), 'collected_objects': 0}


**Задача 9:** Быстрая проверка работы GC

Проверьте, собирает ли GC циклические ссылки.
Верните разницу в количестве объектов до и после сборки.

In [24]:
def task9():
    a = []
    b = []
    a.append(b)
    b.append(a)

    before = len(gc.get_objects())
    collected = gc.collect()
    after = len(gc.get_objects())

    return before - after

print("Задача 9:", task9())

Задача 9: 48


**Задача 10:** Детектор "неубиваемых" объектов

Найдите объекты, которые переживают принудительную сборку мусора.
Выведите количество таких объектов.

In [39]:
def task10():
  res_list = []
  class Persistent:
    def __del__(self):
        res_list.append(self)

  persistent_objs = [Persistent() for _ in range(5)]
  objects_count = \
    len([obj for obj in gc.get_objects() if isinstance(obj, Persistent)])

  del persistent_objs
  collected = gc.collect()

  remaining_objects = \
    len([obj for obj in gc.get_objects() if isinstance(obj, Persistent)])

  return remaining_objects

print("Задача 10:", task10())

Задача 10: 5


In [ ]:
# Небольшая подсказка для создания неудаляемых объектов
class Persistent:
        def __del__(self):
            pass  # Мешает нормальному удалению